<a href="https://colab.research.google.com/github/andrey-de/stepik_de_jun_spark_in_collab/blob/main/SparkTask2_books_and_authors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import sum, desc, max, min, count
from pyspark.sql.functions import year, current_date
from pyspark.sql.functions import year, month, avg

# Создание SparkSession
spark = SparkSession.builder.appName("Read CSV Example").getOrCreate()

In [13]:
# Чтение CSV-файла
df_authors = spark.read.csv("/content/sample_data/authors.csv", header=True, inferSchema=True)
df_books = spark.read.csv("/content/sample_data/books.csv", header=True, inferSchema=True)

In [14]:
# Печать схемы DataFrame
df_authors.printSchema()
df_books.printSchema()

root
 |-- author_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- birth_date: date (nullable = true)
 |-- country: string (nullable = true)

root
 |-- book_id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- author_id: integer (nullable = true)
 |-- genre: string (nullable = true)
 |-- price: double (nullable = true)
 |-- publish_date: date (nullable = true)



In [15]:
df_join = df_authors.join(df_books, on="author_id", how="left")
df_join.printSchema()

root
 |-- author_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- birth_date: date (nullable = true)
 |-- country: string (nullable = true)
 |-- book_id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- price: double (nullable = true)
 |-- publish_date: date (nullable = true)



In [16]:
#result 3.1
df_res1 = df_join.select("author_id", "name", "price").groupBy("author_id").agg(max("name"), sum("price").alias("total_revenue"))
df_res1.orderBy(desc("total_revenue")).show(5)

+---------+---------+------------------+
|author_id|max(name)|     total_revenue|
+---------+---------+------------------+
|        2| Author_2|            231.97|
|        7| Author_7|            132.66|
|        1| Author_1|111.86000000000001|
|        8| Author_8|            107.16|
|        5| Author_5|             88.83|
+---------+---------+------------------+
only showing top 5 rows



In [18]:
#result 3.2
df_res2 = df_join.select("genre").filter("genre IS NOT NULL").groupBy("genre").agg(count("genre").alias("count"))
df_res2.orderBy(desc("count")).show(10)

+-----------+-----+
|      genre|count|
+-----------+-----+
|Non-Fiction|    9|
|    Science|    3|
|    Fiction|    3|
|    Fantasy|    3|
|    Mystery|    2|
+-----------+-----+



In [24]:
#result 3.3
df_res3 = df_join.select("author_id", "name", "price").filter("price IS NOT NULL").groupBy("author_id").agg(max("name").alias("name"), avg("price").alias("average_price"))
df_res3.orderBy(desc("average_price")).show(15)

+---------+---------+-----------------+
|author_id|     name|    average_price|
+---------+---------+-----------------+
|        5| Author_5|            88.83|
|        4| Author_4|             83.7|
|        2| Author_2|          57.9925|
|        9| Author_9|            46.31|
|        7| Author_7|            44.22|
|        6| Author_6|           43.965|
|        1| Author_1|37.28666666666667|
|        8| Author_8|            35.72|
|       10|Author_10|           21.165|
+---------+---------+-----------------+



In [27]:
#result 3.4
df_res4 = df_join.select("author_id", "name", "book_id", "title", "genre", "price", "publish_date").filter("publish_date >= '2000-01-01'")
df_res4.orderBy(desc("price")).show(25)

+---------+---------+-------+-------+-----------+-----+------------+
|author_id|     name|book_id|  title|      genre|price|publish_date|
+---------+---------+-------+-------+-----------+-----+------------+
|        7| Author_7|     20|Book_20|    Mystery|91.48|  2018-12-31|
|        5| Author_5|     19|Book_19|    Science|88.83|  2016-12-31|
|        8| Author_8|     15|Book_15|    Fantasy| 60.0|  2008-12-31|
|        6| Author_6|     17|Book_17|    Fantasy|47.57|  2012-12-31|
|        1| Author_1|     18|Book_18|Non-Fiction|43.92|  2014-12-31|
|        2| Author_2|     16|Book_16|    Fiction|36.22|  2010-12-31|
|       10|Author_10|     11|Book_11|Non-Fiction| 31.7|  2000-12-31|
|        8| Author_8|     12|Book_12|Non-Fiction|31.02|  2002-12-31|
|        1| Author_1|     14|Book_14|    Fiction|26.84|  2006-12-31|
|        8| Author_8|     13|Book_13|Non-Fiction|16.14|  2004-12-31|
+---------+---------+-------+-------+-----------+-----+------------+

